In [2]:
%load_ext autoreload

In [1]:
from QHyper.problems.workflow_scheduling import Workflow, WorkflowSchedulingProblem

In [2]:
from QHyper.solvers.cqm import CQM

<class 'networkx.utils.decorators.argmap'> compilation 5:5: FutureWarning: modularity_matrix will return a numpy array instead of a matrix in NetworkX 3.0.


In [17]:
tasks_file = "/Users/jzawalska/Coding/QHyper/QHyper/problems/workflows_data/workflows/msc_sample_workflow.json"
machines_file = "/Users/jzawalska/Coding/QHyper/QHyper/problems/workflows_data/machines/msc_sample_machines.json"
deadline = 13

In [18]:
workflow = Workflow(tasks_file, machines_file, deadline)
wsp = WorkflowSchedulingProblem(workflow)

OSError: [Errno 22] Invalid argument: 'C:\\Users\\basia\\Desktop\\Praca_Inzynierska\\QHyper\\QHyper\\problems\\workflows\\workflows_data\\workflows\x03_tasks_3_machines_1_path.json'

In [50]:
wsp.workflow.time_matrix

,MachineA,MachineB,MachineD
Task1,4.0,2.0,1.0
Task2,8.0,4.0,2.0
Task3,16.0,8.0,4.0
Task4,32.0,16.0,8.0


In [51]:
wsp.workflow.time_matrix.index[1]

'Task2'

In [52]:
wsp.workflow.cost_matrix

,MachineA,MachineB,MachineD
Task1,1.0,2.0,8.0
Task2,2.0,4.0,16.0
Task3,4.0,8.0,32.0
Task4,8.0,16.0,64.0


In [53]:
print(wsp.objective_function.polynomial)

1.0*x0 + 2.0*x1 + 16.0*x10 + 64.0*x11 + 8.0*x2 + 2.0*x3 + 4.0*x4 + 16.0*x5 + 4.0*x6 + 8.0*x7 + 32.0*x8 + 8.0*x9


In [173]:
wsp.objective_function.as_dict()

{('x0',): 1.0,
 ('x1',): 2.0,
 ('x10',): 16.0,
 ('x11',): 64.0,
 ('x2',): 8.0,
 ('x3',): 2.0,
 ('x4',): 4.0,
 ('x5',): 16.0,
 ('x6',): 4.0,
 ('x7',): 8.0,
 ('x8',): 32.0,
 ('x9',): 8.0}

In [54]:
for sense, constraint in wsp.constraints.items():
    for c in constraint:
        print(c.polynomial, sense, 0)

x0 + x1 + x2 - 1 == 0
x3 + x4 + x5 - 1 == 0
x6 + x7 + x8 - 1 == 0
x10 + x11 + x9 - 1 == 0
4.0*x0 + 2.0*x1 + 16.0*x10 + 8.0*x11 + 1.0*x2 + 8.0*x3 + 4.0*x4 + 2.0*x5 + 32.0*x9 - 13 <= 0
4.0*x0 + 2.0*x1 + 16.0*x10 + 8.0*x11 + 1.0*x2 + 16.0*x6 + 8.0*x7 + 4.0*x8 + 32.0*x9 - 13 <= 0


In [55]:
cqm = wsp.to_cqm()

In [56]:
print(cqm)

Constrained quadratic model: 12 variables, 6 constraints, 42 biases

Objective
  Binary('x0') + 2*Binary('x1') + 16*Binary('x10') + 64*Binary('x11') + 8*Binary('x2') + 2*Binary('x3') + 4*Binary('x4') + 16*Binary('x5') + 4*Binary('x6') + 8*Binary('x7') + 32*Binary('x8') + 8*Binary('x9')

Constraints
  ced060a: -1 + Binary('x0') + Binary('x1') + Binary('x2') == 0.0
  c79b5ef: -1 + Binary('x3') + Binary('x4') + Binary('x5') == 0.0
  c772000: -1 + Binary('x6') + Binary('x7') + Binary('x8') == 0.0
  cfa1aab: -1 + Binary('x10') + Binary('x11') + Binary('x9') == 0.0
  c7ae1d9: -13 + 4*Binary('x0') + 2*Binary('x1') + 16*Binary('x10') + 8*Binary('x11') + Binary('x2') + 8*Binary('x3') + 4*Binary('x4') + 2*Binary('x5') + 32*Binary('x9') <= 0.0
  c6058df: -13 + 4*Binary('x0') + 2*Binary('x1') + 16*Binary('x10') + 8*Binary('x11') + Binary('x2') + 16*Binary('x6') + 8*Binary('x7') + 4*Binary('x8') + 32*Binary('x9') <= 0.0

Bounds



In [57]:
qubo, offset = wsp.to_qubo()

In [58]:
qubo

{('x1', 'x0'): 340.0,
 ('x10', 'x0'): 2560.0,
 ('x10', 'x1'): 1280.0,
 ('x11', 'x0'): 1280.0,
 ('x11', 'x1'): 640.0,
 ('x11', 'x10'): 5140.0,
 ('x2', 'x0'): 180.0,
 ('x2', 'x1'): 100.0,
 ('x2', 'x10'): 640.0,
 ('x2', 'x11'): 320.0,
 ('x3', 'x0'): 640.0,
 ('x3', 'x1'): 320.0,
 ('x3', 'x10'): 2560.0,
 ('x3', 'x11'): 1280.0,
 ('x3', 'x2'): 160.0,
 ('x4', 'x0'): 320.0,
 ('x4', 'x1'): 160.0,
 ('x4', 'x10'): 1280.0,
 ('x4', 'x11'): 640.0,
 ('x4', 'x2'): 80.0,
 ('x4', 'x3'): 660.0,
 ('x5', 'x0'): 160.0,
 ('x5', 'x1'): 80.0,
 ('x5', 'x10'): 640.0,
 ('x5', 'x11'): 320.0,
 ('x5', 'x2'): 40.0,
 ('x5', 'x3'): 340.0,
 ('x5', 'x4'): 180.0,
 ('x6', 'x0'): 1280.0,
 ('x6', 'x1'): 640.0,
 ('x6', 'x10'): 5120.0,
 ('x6', 'x11'): 2560.0,
 ('x6', 'x2'): 320.0,
 ('x7', 'x0'): 640.0,
 ('x7', 'x1'): 320.0,
 ('x7', 'x10'): 2560.0,
 ('x7', 'x11'): 1280.0,
 ('x7', 'x2'): 160.0,
 ('x7', 'x6'): 2580.0,
 ('x8', 'x0'): 320.0,
 ('x8', 'x1'): 160.0,
 ('x8', 'x10'): 1280.0,
 ('x8', 'x11'): 640.0,
 ('x8', 'x2'): 80.0,
 (

In [59]:
offset

3420.0

In [60]:
cqm_problem = CQM(problem=wsp, time=5)

In [61]:
solution = cqm_problem.solve()
print(solution)

{'x0': 0.0, 'x1': 0.0, 'x10': 0.0, 'x11': 1.0, 'x2': 1.0, 'x3': 0.0, 'x4': 1.0, 'x5': 0.0, 'x6': 0.0, 'x7': 0.0, 'x8': 1.0, 'x9': 0.0}


In [62]:
wsp.decode_solution(solution)

{'Task4': 'MachineD',
 'Task1': 'MachineD',
 'Task2': 'MachineB',
 'Task3': 'MachineD'}

In [167]:
%autoreload 2

In [168]:
from QHyper.solvers.gurobi.gurobi import Gurobi

In [169]:
gurobi = Gurobi(problem=wsp)

In [170]:
solution_gurobi = gurobi.solve()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 12 columns and 48 nonzeros
Model fingerprint: 0xa6344de6
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 108.0000000
Presolve removed 8 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 108 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.080000000000e+02, best bound 1.080000000000e+02, gap 0.0000%


In [171]:
solution_gurobi

{'x0': 0.0,
 'x1': 0.0,
 'x2': 1.0,
 'x3': 0.0,
 'x4': 1.0,
 'x5': 0.0,
 'x6': 0.0,
 'x7': 0.0,
 'x8': 1.0,
 'x9': 0.0,
 'x10': 0.0,
 'x11': 1.0}

In [172]:
wsp.decode_solution(solution_gurobi)

{'Task1': 'MachineD',
 'Task2': 'MachineB',
 'Task3': 'MachineD',
 'Task4': 'MachineD'}